In [1]:
import numpy as np
import napari
# from tifffile import imread, imwrite
import tifffile
import glob
import os
import cv2
from PIL import Image
from matplotlib import image
import ntpath
from matplotlib import pyplot as plt
from skimage import morphology
import os
import collections
from tqdm import tqdm
import ntpath

In [2]:
# the new label to be combined to old labels after hand segmenting
additional_label = 9

In [3]:
#path to hand segmentation tif file
file_path = "/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/masks/roi1_nuc_00.tif"
name = ntpath.basename(file_path.split('.')[0])
print(name)

label_path = "/data/FIBSEM/vrinda/ROI_1/test_code/res/Apr_20/test/run_3/chromosome_all_labels_run_3.tiff"

#path to the folder to save output - combined new labels
# path = "/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/output/nuc_00/apr_20/combined_labels_"

#path to output folder with individual chromosomes
output_path = "/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/output/nuc_00/apr_27_ip_clean_600_op_clean_400_for_4"

roi1_nuc_00


In [4]:
def sort_list(lst):
    lst = sorted(lst, key = lambda fl : int(ntpath.basename(fl).split('.')[0]))

    return lst

def clean_small_obj(im, ob_size):
    cleaned_im = np.copy(im)
    cleaned_index = (cleaned_im!=0)

    img = cleaned_index
    img = morphology.remove_small_objects(img, ob_size, connectivity=1)
    target_img = cleaned_im[:,:]

    cleaned_im[:,:] = np.multiply(target_img, img)

    return cleaned_im

def load_image(img_array, n):
    img_temp = Image.open(img_array[n])
    img_temp.load()
    img_temp = np.array(img_temp)
    
    return img_temp

def find_contours(im):
    contours, hierarchy = cv2.findContours(image=im, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    mask = np.zeros(im.shape, np.uint8)

    return contours, hierarchy, mask

# pass points as tuples
def cal(p1,p2):
  return np.linalg.norm(np.array(p1) - np.array(p2))

In [5]:
original=[]
chromo=[]
pattern=[]

for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/centroid/*.jpg"):
  original.append(file)
for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/chro_nucl_00_000_650/*.jpg"):
  chromo.append(file)
for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/patt/*.jpg"):
  pattern.append(file)

original = sort_list(original)
chromo = sort_list(chromo)
pattern = sort_list(pattern)
print(original[0])
print(chromo[0])
print(pattern[0])

/data/FIBSEM/vrinda/ROI_1/centroid/00000.jpg
/data/FIBSEM/vrinda/ROI_1/chro_nucl_00_000_650/000.jpg
/data/FIBSEM/vrinda/ROI_1/patt/000.jpg


In [6]:
img = tifffile.imread(file_path)
print(len(img))
label = tifffile.imread(label_path)
print(len(label))

651
651


In [7]:
#combine hand segmented label
new_label_stack={}
new_label_list=[]
# temp2_before=[]
# temp2_after=[]
# for k in tqdm(range(269,279)):
for k in tqdm(range(len(img))):
    hand_label = img[k]
    old_label = label[k]
    chr = load_image(chromo,k)
    chr = clean_small_obj(chr,800)
#     chr[chr<40]=0
    
    new_label = np.array([])
    new_label = np.copy(old_label)   
    unique_hand_label = np.unique(hand_label)
    
    if additional_label in unique_hand_label:
        temp = (hand_label==additional_label)
        temp = temp.astype(np.uint8)
        temp2= cv2.bitwise_and(chr,chr, mask = temp)
    
#         print(np.unique(temp2))
#         print(type(temp2))
#         temp2_before.append(temp2)
        temp2[temp2<40]=0
        temp2[(temp2!=0)]=additional_label
#         temp2_after.append(temp2)
        new_label[np.where(temp2==additional_label)] = additional_label
        print("new_label_combined")
    else:
#         new_mask = np.zeros(img[k].shape, np.uint8)
#         temp2_before.append(new_mask)
        print("label_9_not_found_in this_slice")
#         temp2_after.append(new_mask)

    new_label_stack[k]=new_label
    new_label_list.append(new_label)
    
print("combined_all_hand_segmented_labels")        
new_label_list = np.array(new_label_list)        
# temp2_after = np.array(temp2_after)
# temp2_before = np.array(temp2_before)

# new_path = "/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/output/nuc_00/apr_25_clean_small/"
# temp2_after_path = '/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/output/nuc_00/temp2_after.tiff'
# temp2_before_path = '/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/output/nuc_00/temp2_before.tiff'
if not os.path.exists(output_path):
    from pathlib import Path
    Path(output_path).mkdir(parents=True, exist_ok=True)     
ch_path = output_path +'/chromosome_combined_label_roi1_nuc00.tiff'
tifffile.imwrite(ch_path, new_label_list)   
# tifffile.imwrite(temp2_after_path, temp2_after) 
# tifffile.imwrite(temp2_before_path, temp2_before) 
print("tiff file saved")

  0%|▍                                                                                                                                               | 2/651 [00:00<00:55, 11.63it/s]

label_9_not_found_in this_slice
label_9_not_found_in this_slice
label_9_not_found_in this_slice



  1%|▉                                                                                                                                               | 4/651 [00:00<00:51, 12.57it/s]

label_9_not_found_in this_slice
label_9_not_found_in this_slice



  1%|█▎                                                                                                                                              | 6/651 [00:00<00:50, 12.82it/s]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 648/651 [00:55<00:00, 13.47it/s]

label_9_not_found_in this_slice
label_9_not_found_in this_slice
label_9_not_found_in this_slice


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 651/651 [00:55<00:00, 11.72it/s]

label_9_not_found_in this_slice
label_9_not_found_in this_slice
combined_all_hand_segmented_labels


tiff file saved


In [8]:
avoid=[43,50]


In [9]:
#specify labels to be extracted and saved as individual folders
labels_to_be_extracted = [1, 2, 3, 4, 7, 9]

chro_map={}
for k in range(len(img)):
    if k in avoid:
        chro_map[k]='inf'
    else:
        chro_map[k]={}

for label in labels_to_be_extracted:
    for k in range(len(img)):
        temp_mask = (new_label_stack[k]==label)
        temp_mask = temp_mask.astype(np.uint8)
        
        cont, hier, mask = find_contours(temp_mask)
        for i in range(len(cont)):
            if cv2.arcLength(cont[i],True) > 25 :
                M = cv2.moments(cont[i])
                cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
                chro_map[k][(cX,cY)]=label
        
        org = load_image(original,k)
        extracted = cv2.bitwise_and(org,org, mask = temp_mask)
        if label == 4:
            extracted = clean_small_obj(extracted,400)
#         plt.imshow(extracted)
        final_output_path = output_path + "/individual/chromosome/ch_" + str(label)
        if not os.path.exists(final_output_path):
            from pathlib import Path
            Path(final_output_path).mkdir(parents=True, exist_ok=True)
        cv2.imwrite(final_output_path+"/"+ str(k)+".jpg", (extracted))
        
    print(label)
                    
# print(chro_map)


1
2
3
4
7
9


Below part is to find the corersponding SC's and save as individual SC folder according to the current chormosome labels


In [10]:
pattern_img={}
pattern_avoid=set()

In [11]:
def find_pattern_label(X,Y,chro_blobs_in_the_slice):
  closest_id, closest_dist = -1,float('inf')
  for pt,id in chro_blobs_in_the_slice.items():
    # if centroid_prev == 0:
    #   continue
    dist = cal((X,Y), pt)
#     print((X,Y), pt, dist)
    if closest_dist > dist:
        closest_dist, closest_pt, closest_id = dist,pt,id
  return closest_id

In [12]:
def find_patterns(chro_map):
    zero_moment=[]
    pattern_img_list=[]
    for k in tqdm(range(len(pattern))):
#         print(k)
        patt = load_image(pattern,k)
        patt = clean_small_obj(patt,600)
        patt[patt<40]=0
        contours_p, hier_p, mask_p = find_contours(patt)
        n = ntpath.basename(pattern[k]).split('.')[0]
        n=int(n)
        if n in avoid:
          print(n)
          pattern_img[k] = mask_p
          pattern_img_list.append(mask_p)  
          pattern_avoid.add(k)
          continue
        if len(contours_p)==0:
          pattern_avoid.add(k)
          print("pattern_avoid", pattern_avoid)
          pattern_img[k] = mask_p
          pattern_img_list.append(mask_p)  
          continue
        pattern_map_inside_find_patterns = {}
        blobs_in_slice = chro_map[k]
        #     print("blobs_in_slice", blobs_in_slice)
        for i in range(len(contours_p)):
          if cv2.arcLength(contours_p[i],True) > 25 :
            M = cv2.moments(contours_p[i])
            if M["m00"] == 0:
              zero_moment.append(k)
              continue
            cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
            patt_label = find_pattern_label(cX,cY, blobs_in_slice)
        #         print("patt_label", (cX,cY), patt_label)
            pattern_map_inside_find_patterns[(cX,cY)] = [patt_label]
            points_new=[]
            for l in range(len(contours_p[i])):
              points_new.append(contours_p[i][l][0])
            points_new = np.array(points_new)
            cv2.drawContours(mask_p,[points_new],-1,patt_label, thickness = cv2.FILLED)
        pattern_img[k]=mask_p
        
    print("zero_moment", zero_moment)
    pattern_img_list.append(mask_p)
    pattern_img_list = np.array(pattern_img_list)   
    
    if not os.path.exists(output_path):
        from pathlib import Path
        Path(output_path).mkdir(parents=True, exist_ok=True)     
    patt_path = output_path +'/pattern_combined_label_roi1_nuc00.tiff'
    tifffile.imwrite(patt_path, pattern_img_list)   
    print("tiff file saved")


In [13]:
find_patterns(chro_map)

  7%|█████████▉                                                                                                                                     | 45/651 [00:01<00:28, 21.10it/s]

43


  8%|███████████▋                                                                                                                                   | 53/651 [00:02<00:34, 17.21it/s]

50


 36%|███████████████████████████████████████████████████▍                                                                                          | 236/651 [00:12<00:20, 19.94it/s]

...

pattern_avoid {285, 43, 50, 315, 316, 317, 320, 321, 322, 324, 325, 477, 478, 479, 480, 231, 233, 235, 236, 237, 238, 239, 241, 243, 244}


 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                   | 490/651 [00:24<00:10, 15.71it/s]

pattern_avoid {285, 43, 50, 315, 316, 317, 320, 321, 322, 324, 325, 477, 478, 479, 480, 231, 488, 233, 235, 236, 237, 238, 239, 241, 243, 244}
pattern_avoid {285, 43, 50, 315, 316, 317, 320, 321, 322, 324, 325, 477, 478, 479, 480, 231, 488, 233, 489, 235, 236, 237, 238, 239, 241, 243, 244}
pattern_avoid {285, 43, 50, 315, 316, 317, 320, 321, 322, 324, 325, 477, 478, 479, 480, 231, 488, 233, 489, 235, 236, 237, 238, 239, 490, 241, 243, 244}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 651/651 [00:32<00:00, 19.94it/s]


zero_moment []
tiff file saved


In [14]:
for label in labels_to_be_extracted:
    for k in range(len(img)):
        temp_mask = (pattern_img[k]==label)
        temp_mask = temp_mask.astype(np.uint8)
        
        cont, hier, mask = find_contours(temp_mask)
        for i in range(len(cont)):
            if cv2.arcLength(cont[i],True) > 25 :
                M = cv2.moments(cont[i])
                cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
#                 chro_map[k][(cX,cY)]=label
        
        org = load_image(original,k)
        extracted = cv2.bitwise_and(org,org, mask = temp_mask)
#         plt.imshow(extracted)
        final_output_path = output_path + "/individual/pattern/patt_" + str(label)
        if not os.path.exists(final_output_path):
            from pathlib import Path
            Path(final_output_path).mkdir(parents=True, exist_ok=True)
        cv2.imwrite(final_output_path+"/"+ str(k)+".jpg", (extracted))
        
    print(label)
                  

1
2
3
4
7
9
